In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install transformers

In [ ]:
import random

import numpy as np
import torch.nn as nn
from tqdm import tqdm
import time
import datetime

import os
import numpy as np
import pandas as pd
import re                                  
import string  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
from transformers import BertModel, BertConfig

from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [ ]:
import zipfile



# Will unzip the files so that you can see them..
with zipfile.ZipFile("../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip","r") as z:
    z.extractall(".")

In [ ]:
train_df1=pd.read_csv("./train.csv")
train_df1.head()

In [ ]:
class Settings:
#     PROJ_NAME = 'Jigsaw-Toxic-Comment-Classification'
#     root_path = os.getcwd().split(PROJ_NAME)[0] + "/" + PROJ_NAME + "/"
    MAX_LEN = 512
    TRAIN_BATCH_SIZE = 16
    VALID_BATCH_SIZE = 16
    EPOCHS = 1
    RANDOM_STATE = 42
    MODEL_PATH = 'toxic_model.bin'
    TRAIN_NUM_WORKERS = 4
    VAL_NUM_WORKERS = 1

    # training data directory
#     TRAIN_DATA = root_path + "/training/data/train.csv"

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_dim = 768
    hidden_dim = 50
    output_dim = 6
    bert_model_name = 'bert-base-uncased'

    # mapping of columns
    columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
class BERTDataset:
    def __init__(self, texts, targets):
        self.settings = Settings
        self.texts = texts
        self.targets = targets
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                                       do_lower_case=True)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        sent = str(self.texts[item])
        inputs = self.tokenizer.encode_plus(
            text=sent,
            add_special_tokens=True,
            max_length=self.settings.MAX_LEN,
            pad_to_max_length = True,
            return_attention_mask=True
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        return {
            'input_ids': torch.tensor(ids),
            'attention_mask': torch.tensor(mask),
            'token_type_ids': torch.tensor(token_type_ids),
            'targets': torch.tensor(self.targets[item])
        }

In [ ]:

class Engine:
    def __init__(self):
        pass

    def loss_fn(self, outputs, targets):
        return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 6))

    def accuracy_threshold(self, y_pred, y_true, thresh: float = 0.5, sigmoid: bool = True):
        if sigmoid:
            y_pred = y_pred.sigmoid()
        return ((y_pred > thresh) == y_true.byte()).float().mean().item()

    def set_seed(self, seed_value=42):
        random.seed(seed_value)
        np.random.seed(seed_value)
        torch.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)

    def train_fn(self, data_loader, model, optimizer, device, schedular):
        print("Starting training...\n")
        # Reset the total loss for this epoch.
        total_loss, batch_loss, batch_counts = 0, 0, 0
        t0_epoch, t0_batch = time.time(), time.time()
        model.train()
        for step, data in tqdm(enumerate(data_loader), total=len(data_loader)):
            batch_counts += 1
            b_input_ids = data['input_ids']
            b_attn_mask = data['attention_mask']
            b_labels = data['targets']

            # moving tensors to device
            b_input_ids = b_input_ids.to(device,dtype=torch.long)
            b_attn_mask = b_attn_mask.to(device,dtype=torch.long)
            b_labels = b_labels.to(device,dtype=torch.float)

            # optimizer.zero_grad()

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because
            # accumulating the gradients is "convenient while training RNNs".
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()

            logits = model(b_input_ids, b_attn_mask)

            loss = self.loss_fn(logits, b_labels.float())
            batch_loss += loss.item()
            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value
            # from the tensor.
            total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()
            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc
            optimizer.step()
            # Update the learning rate
            schedular.step()

            if step % 500 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = self.format_time(time.time() - t0_epoch)
                # Report progress.
                print(' \n Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(data_loader), elapsed))

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_loss / len(data_loader)
        # Measure how long this epoch took.
        training_time = self.format_time(time.time() - t0_epoch)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))

    def eval_fn(self, data_loader, model, device):
        print("Starting evaluation...\n")
        t0 = time.time()
        model.eval()
        val_accuracy = []
        val_loss = []
        with torch.no_grad():
            for step, data in tqdm(enumerate(data_loader), total=len(data_loader)):
                b_input_ids = data['input_ids']
                b_attn_mask = data['attention_mask']
                b_labels = data['targets']

                # moving tensors to device
                b_input_ids = b_input_ids.to(device,dtype=torch.long)
                b_attn_mask = b_attn_mask.to(device,dtype=torch.long)
                b_labels = b_labels.to(device,dtype=torch.float)

                logits = model(b_input_ids, b_attn_mask)
                loss = self.loss_fn(logits, b_labels.float())
                val_loss.append(loss.item())
                accuracy = self.accuracy_threshold(logits.view(-1, 6), b_labels.view(-1, 6))
                val_accuracy.append(accuracy)

        val_loss = np.mean(val_loss)
        val_accuracy = np.mean(val_accuracy)
        validation_time = self.format_time(time.time() - t0)

        print("  Average Validation Loss: {0:.2f}".format(val_loss))
        print("  Average Validation Accuracy: {0:.2f}".format(val_accuracy))
        print("  Validation took: {:}".format(validation_time))

        return val_loss, val_accuracy

    def format_time(self, elapsed):
        '''
        Takes a time in seconds and returns a string hh:mm:ss
        '''
        # Round to the nearest second.
        elapsed_rounded = int(round(elapsed))

        # Format as hh:mm:ss
        return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, freeze_bert=False):
        super(BERTClassifier, self).__init__()
        self.settings = Settings
        self.input_dim = self.settings.input_dim
        self.hidden_dim = self.settings.hidden_dim
        self.output_dim = self.settings.output_dim

        # loading the bert model
        self.bert = BertModel.from_pretrained(self.settings.bert_model_name)

        # adding custom layers according to the problem statement
        self.classifier = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, self.output_dim)
        )

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)

        last_hidden_state_cls = outputs[0][:, 0, :]
        logits = self.classifier(last_hidden_state_cls)

        return logits

    # def print_model_details(self):
    #     # Get all of the model's parameters as a list of tuples.
    #     params = list(self.bert.named_parameters())
    #
    #     print('The BERT Base Uncased Model Has {:} different named parameters.\n'.format(len(params)))
    #
    #     print('==== Embedding Layer ====\n')
    #
    #     for p in params[0:5]:
    #         print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
    #
    #     print('\n==== First Transformer ====\n')
    #
    #     for p in params[5:21]:
    #         print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
    #
    #     print('\n==== Output Layer ====\n')
    #
    #     for p in params[-4:]:
    #         print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
class Preprocess:
    def __init__(self):
        pass

    def clean_text(self, text):
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
        return text

In [ ]:
class Train:
    def __init__(self):
        # initialize required class
        self.settings = Settings
        self.engine = Engine()
        self.preprocess = Preprocess()

        # initialize required variables
        self.bert_classifier = None
        self.optimizer = None
        self.scheduler = None
        self.train_data_loader = None
        self.val_data_loader = None
        self.total_steps = None
        self.best_accuracy = 0

    def __initialize(self):
        # Instantiate Bert Classifier
        self.bert_classifier = BERTClassifier(freeze_bert=False)
        self.bert_classifier.to(self.settings.DEVICE)

        # Create the optimizer
        self.optimizer = AdamW(self.bert_classifier.parameters(),
                               lr=5e-5,  # Default learning rate
                               eps=1e-8  # Default epsilon value
                               )
        # Set up the learning rate scheduler
        self.scheduler = get_linear_schedule_with_warmup(self.optimizer,
                                                         num_warmup_steps=0,  # Default value
                                                         num_training_steps=self.total_steps)

    def crete_data_loaders(self, dataset):
        pass

    def load_data(self):
        train_df = train_df1.fillna("none")
        train_df['comment_text'] = train_df['comment_text'].apply(lambda x: self.preprocess.clean_text(x))
        X = list(train_df['comment_text'])
        y = np.array(train_df.loc[:, 'toxic':])

        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=self.settings.RANDOM_STATE)

        # training dataset
        train_dataset = BERTDataset(X_train, y_train)

        # validation dataset
        val_dataset = BERTDataset(X_val, y_val)

        self.train_data_loader = DataLoader(train_dataset,
                                            batch_size=self.settings.TRAIN_BATCH_SIZE,
                                            shuffle=True,
                                            num_workers=self.settings.TRAIN_NUM_WORKERS)

        self.val_data_loader = DataLoader(val_dataset,
                                          batch_size=self.settings.VALID_BATCH_SIZE,
                                          shuffle=True,
                                          num_workers=self.settings.VAL_NUM_WORKERS)

        self.total_steps = int(len(X_train) / self.settings.TRAIN_BATCH_SIZE * self.settings.EPOCHS)

    def train(self):
        for epochs in range(self.settings.EPOCHS):
          print("\n")
          print("-"*70)
          print("Epoch--- ",str(epochs))
          print("-"*70)
          print("\n")

          # calling the training function in engine.py file
          self.engine.train_fn(data_loader=self.train_data_loader,
                                model=self.bert_classifier,
                                optimizer=self.optimizer,
                                device=self.settings.DEVICE,
                                schedular=self.scheduler)

          # calling the evaluation function from the engine.py file to compute evaluation
          val_loss, val_accuracy = self.engine.eval_fn(data_loader=self.val_data_loader,
                                                        model=self.bert_classifier,
                                                        device=self.settings.DEVICE)

          # updating the accuracy
          if val_accuracy > self.best_accuracy:
              torch.save(self.bert_classifier.state_dict(), self.settings.MODEL_PATH)
              self.best_accuracy = val_accuracy

    def run(self):
        try:
            print("Loading and Preparing the Dataset-----!! ")
            self.load_data()
            print("Dataset Successfully Loaded and Prepared-----!! ")
            print()
            print("-" * 70)
            print("Loading and Initializing the Bert Model -----!! ")
            self.__initialize()
            print("Model Successfully Loaded and Initialized-----!! ")
            print()
            print("-" * 70)
            print("------------------Starting Training-----------!!")
            self.engine.set_seed()
            self.train()
            print("Training complete-----!!!")

        except BaseException as ex:
            print("Following Exception Occurred---!! ", str(ex))


In [ ]:
t =Train()
t.run()

## Inference Code

In [ ]:
test_df = pd.read_csv('./test.csv')
test_df

In [ ]:
MODEL = BERTClassifier()
MODEL.load_state_dict(torch.load(Settings.MODEL_PATH,map_location=torch.device(Settings.DEVICE)))
MODEL.to(Settings.DEVICE)

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
test_df['comment_text'] = test_df['comment_text'].apply(lambda x: clean_text(x))

In [ ]:
def bert_predict(model, test_dataloader):
    model.eval()
    all_logits = []
    for batch in test_dataloader:
        b_input_ids, b_attn_mask = tuple(t.to(Settings.DEVICE) for t in batch)[:2]
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    all_logits = torch.cat(all_logits, dim=0)
    probs = all_logits.sigmoid().cpu().numpy()
    
    return probs

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
def preprocessing_for_bert(data):
    input_ids = []
    attention_masks = []
    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            text = sent,
            add_special_tokens = True,
            max_length = Settings.MAX_LEN,
            pad_to_max_length = True,
            return_attention_mask = True
        )
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    
    return input_ids, attention_masks

In [ ]:
test_inputs, test_masks = preprocessing_for_bert(test_df.comment_text)
test_dataset = TensorDataset(test_inputs, test_masks)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=32)

In [ ]:
probs = bert_predict(MODEL, test_dataloader)

In [ ]:
submission = pd.DataFrame(probs,columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate'])
test_df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]=submission
final_sub = test_df[['id','toxic','severe_toxic','obscene','threat','insult','identity_hate']]
final_sub.head()
                    

In [ ]:
final_sub.to_csv('submission.csv', index=False)